# Get Sentinel-3 SST
Explain how to obtain netCDF files usin EUMDAC.     
One needs to have EUMDAC credentials.

In [38]:
import sys
sys.path.insert(0, '..')
import os
from filament import filament
import datetime
import shutil
import eumdac
import glob
import logging
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from importlib import reload
reload(filament)
logger = logging.getLogger("SST-Sentinel3")
logger.setLevel(logging.INFO)
logging.info("Starting")
plt.style.use("filament.mplstyle")

## Directories

In [22]:
# Create a download directory for our downloaded products
download_dir = "/home/ctroupin/data/Sentinel3/"
figdir = "../figures/SST/BloomJune2024/"
os.makedirs(download_dir, exist_ok=True)

## Get access token

In [23]:
with open("/home/ctroupin/.eumdac/credentials") as df:
    theinfo = df.readline()
    token = eumdac.AccessToken((theinfo.split(",")[0], theinfo.split(",")[1]))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '18d9fe86-30d2-32e3-886c-7d50f2600644' expires 2024-06-28 16:08:51.425066


Show only the products corresponding to Sentinel-3 SST

In [11]:
for collectionID in datastore.collections:
    if ('Sentinel-3' in collectionID.title) & ('SST' in collectionID.title):
        print(f"{collectionID}: {collectionID.title}")

EO:EUM:DAT:0412: SLSTR Level 2 Sea Surface Temperature (SST) - Sentinel-3
EO:EUM:DAT:0582: SLSTR Level 2 Sea Surface Temperature (SST) (version BC003) - Sentinel-3 - Reprocessed


Select the collection of interest

In [12]:
collectionID = 'EO:EUM:DAT:0412'

# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

SLSTR Level 2 Sea Surface Temperature (SST) - Sentinel-3


Get product based on the time period and the region of interest

In [36]:
start = datetime.datetime(2024, 6, 20)
end = datetime.datetime(2024, 6, 28)
domain = filament.regions["canarydomain"]

myproj = ccrs.Mercator(central_longitude=0.5 * (domain[0] + domain[1]), min_latitude=domain[2], 
                       max_latitude=domain[3])

roi = [[domain[0], domain[2]], [domain[1], domain[2]], [domain[1], domain[3]], 
       [domain[0], domain[3]], [domain[0], domain[2]]]

WKT = 'POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi]))

products = selected_collection.search(
    geo=WKT,
    dtstart=start, 
    dtend=end)

## Perform download

In [43]:
outputfile

'/home/ctroupin/data/Sentinel3/S3A_SL_2_WST____20240626T104909_20240626T105209_20240626T124526_0179_114_051_2520_MAR_O_NR_003.SEN3.zip'

In [42]:
for product in products:
    selected_product = datastore.get_product(product_id=str(product), collection_id=collectionID)

    with selected_product.open() as fsrc:
        outputfile = os.path.join(download_dir, fsrc.name)

        if os.path.isfile(outputfile):
            logger.info(f"{outputfile} already downloaded")
        else:
            with open(outputfile, mode='wb') as fdst:
                logger.info(f'Downloading {fsrc.name}')
                shutil.copyfileobj(fsrc, fdst)
                logger.info(f'Download of product {fsrc.name} finished.')

Download of product S3A_SL_2_WST____20240627T231452_20240627T231752_20240628T013919_0180_114_073_0360_MAR_O_NR_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240627T223602_20240627T223902_20240628T005953_0179_094_315_0360_MAR_O_NR_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240627T112507_20240627T112807_20240627T132031_0179_094_308_2520_MAR_O_NR_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240627T112207_20240627T112507_20240627T132031_0179_094_308_2340_MAR_O_NR_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240626T230212_20240626T230512_20240627T012553_0179_094_301_0360_MAR_O_NR_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240626T223055_20240627T001154_20240628T044824_6059_094_300______MAR_O_NT_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240626T115118_20240626T115418_20240626T134607_0179_094_294_2520_MAR_O_NR_003.SEN3.zip finished.
Download of product S3B_SL_2_WST____20240626T114818_20240626T1


KeyboardInterrupt



## Loop on files and make plots

In [47]:
datafilelist = sorted(glob.glob(os.path.join(download_dir, "20240626230212-MAR-L2P_GHRSST-SSTskin-SLSTRB-202406*.nc")))
datafilelist

['/home/ctroupin/data/Sentinel3/20240626230212-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240627012553-v02.0-fv01.0.nc']

In [51]:
for ifile, datafile in enumerate(datafilelist):
    logger.info("Working on {}".format(datafile))
    
    sst = filament.SST()
    sst.read_from_sentinel3(datafile)
    sst.field = np.ma.masked_where(sst.qflag < 1, sst.field)    
    titletext = sst.get_title()
    

    sstmin, sstmax = 17.5, 22.5
    
    figname = os.path.join(figdir, sst.get_figname())

    fig = plt.figure(figsize=(12, 10))
    ax = plt.subplot(111, projection=myproj)
    ax.set_extent(domain)
    ax.set_title(titletext, fontsize=24)
    
    pcm = ax.pcolormesh(sst.lon.data, sst.lat.data, sst.field, cmap=plt.cm.RdYlBu_r, 
                        vmin=sstmin, vmax=sstmax, transform=filament.datacrs)
    
    cb = plt.colorbar(pcm, extend="both", shrink=0.75)
    cb.set_label("°C", rotation=0, ha="left")

    iii = 0        
    #vis.add_to_plot(ax, myproj)

    ax.add_feature(filament.coast_f, linewidth=.2, color=".5")
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=.5, color='.25', linestyle='--', zorder=2)
    gl.top_labels = False
    gl.right_labels = False

    plt.savefig(figname)
    # plt.show()
    plt.close(fig)

INFO:SST-Sentinel3:Working on /home/ctroupin/data/Sentinel3/20240626230212-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240627012553-v02.0-fv01.0.nc
